In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from pathlib import Path
import sympy as sp


In [ ]:
rdir = '../results_stage2_chunk/'
datadir = '../experiment/data/stage2/data/'

In [ ]:
frames = []
i = 0
for f in Path(rdir).rglob('*.json'):
#     print(f)
    if '7_7' in str(f):
        continue
    if '135' not in str(f):
        continue
    if 'gpzgd' in str(f):
        continue
    with open(f, 'r') as of:
        d = json.load(of)
    frames.append(d)
    i += 1
    
print('loaded',i,'results')
df = pd.DataFrame.from_records(frames)

# hide linear regression name
df.loc[df.algorithm=='LinearRegression','algorithm'] = 'Mamba'
# fix cutoff dataset names
df['dataset'] = df['dataset'].apply(lambda x: x+'ata' if x.endswith('_d') else x)
########################################
# normalize simplicity
df['simplicity'] = (df['simplicity']-df['simplicity'].min())/(df['simplicity'].max()-df['simplicity'].min())
########################################
# time transform
df['time_hr'] = df['time_time']/3600
df['time_mins'] = df['time_time']/60
########################################
# deconstruct dataset names
df['dataset-full'] = df['dataset'].copy()
df['dataset'] = df['dataset'].apply(lambda x: '_'.join(x.split('_')[1:-1]))
df['task'] = df['dataset'].apply(lambda x: x.split('value_')[-1].split('_')[0])
df['horizon'] = df['dataset'].apply(lambda x: 7 if '7' in x else 14)
df['random_state'] = df['dataset-full'].apply(lambda x: x.split('_')[0])
df.head()

In [ ]:
METRICS = [
   'mse_train', 
   'mae_train', 
   'r2_train',
   'mse_test', 
   'mae_test', 
   'r2_test', 
   'accuracy', 
   'feature_absence_score' 
]

In [ ]:
df.columns

In [ ]:
df['task'].unique()

# check run completion

In [ ]:
df.groupby(['dataset','algorithm'])['random_state'].count().unstack()

In [ ]:
order = df.groupby(['algorithm'])['r2_test'].mean().sort_values(ascending=False).index
df.groupby(['task','horizon','algorithm'])['r2_test'].median().unstack().round(3)[order]

In [ ]:
df.groupby(['algorithm'])['r2_test'].median().sort_values(ascending=False).round(3)

In [ ]:
order = df.groupby(['algorithm'])['simplicity'].mean().sort_values(ascending=False).index
df.groupby(['task','horizon','algorithm'])['simplicity'].mean().unstack().round(3)[order]

In [ ]:
# df.groupby(['algorithm'])['simplicity'].mean().sort_values(ascending=False).round(3)

In [ ]:
# model_sel
import pdb
median_scores= df.groupby(['task','horizon','algorithm'])['r2_test'].median().unstack()
sel_model = {}
frames = []
for idx, row in median_scores.iterrows():
#     task = row['task'], row['horizon'], row['algorithm'], 
    for alg in row.index:
        if np.isnan(row[alg]): continue
        dfg = df.loc[
            (df.task==idx[0])
            & (df.horizon==idx[1])
            & (df.algorithm==alg)
        ]
        entry = dfg.loc[(dfg.r2_test-row.loc[alg]).abs().idxmin()]
        assert isinstance(entry, pd.Series)
        frames.append(entry.to_dict())
df_best = pd.DataFrame.from_records(frames)

In [ ]:
df_best.groupby('algorithm').count()

In [ ]:
df_best.loc[df_best.algorithm=='QLattice','symbolic_model']

In [ ]:
# redo models

In [ ]:
from evaluation import get_symbolic_model, simplicity, round_floats

def redo_model(x):
    seed = x['random_state'] #.values[0]
    ds = x['dataset'] #.values[0]
    task = x['task'] #.values[0]
    dataset = pd.read_csv(f'../experiment/data/stage2/data/{seed}_{ds}_train.csv')
#     '../experiment/data/stage2/data/'
#     print(f'../experiment/data/stage2/data/{seed}_{ds}_{task}_train.csv')
    feature_names = [k for k in dataset.columns if k!= task]
    local_dict = {k:sp.Symbol(k) for k in feature_names}
    if x['algorithm'] == 'QLattice':
        print('symbolic_model:',
              x.symbolic_model)
    mdl = str(get_symbolic_model(x.symbolic_model, local_dict=local_dict, simplify=False))
    simp = simplicity(mdl,feature_names,simplify=False)
    if x['algorithm'] == 'QLattice':
        print('get_symbolic_model:',
              mdl)
    x['simplified_model'] = round_floats(mdl, 6)
    x['simplicity'] = simp
#     if x['algorithm'] == 'QLattice': pdb.set_trace()
    return x
   
df_best = df_best.transform(lambda x: redo_model(x), axis=1) 
# normalize simplicity
df_best['simplicity'] = (df_best['simplicity']-df_best['simplicity'].min())/(df_best['simplicity'].max()-df_best['simplicity'].min())

df_best[['task','simplified_model','simplicity','algorithm']]

# TODO
- for each dataset 
    - for each algorithm
        - report R2, mse, simplicity
        - report simplified_model
        - plot test predictions versus real data

In [ ]:
dfm = df_best[['algorithm','task','r2_test','simplicity']].melt(id_vars = ['algorithm','task'])
display(
    dfm.loc[dfm.variable=='r2_test'].groupby(['algorithm','task'])['value'].median().round(2).unstack()
)
display(
    dfm.loc[dfm.variable=='simplicity'].groupby(['algorithm','task'])['value'].median().round(2).unstack()
)

In [ ]:
%config InlineBackend.figure_format ='retina'
from sklearn.metrics import r2_score
import sympy as sp 
import pdb 
from matplotlib import rc
from evaluation import get_symbolic_model, round_floats
rc('text', usetex=True)
rc('font', family='serif')
plt.rcParams.update({"text.latex.preamble":r'\usepackage{tabularx} '})

def comparison_plot(task,horizon,alg,dfg):
    """Comparison plot of model for report."""
    seed = dfg['random_state'].values[0]
    ds = dfg['dataset'].values[0]
#     '../experiment/data/stage2/data/'
    print(f'../experiment/data/stage2/data/{seed}_{ds}_{task}_train.csv')
    df1 = pd.read_csv(f'../experiment/data/stage2/data/{seed}_{ds}_train.csv')
    df1['fold'] = 'train'
    df2 = pd.read_csv(f'../experiment/data/stage2/data/{seed}_{ds}_test.csv')
    df2['fold'] = 'test'
    df = df1.append(df2)
    df['date'] = df['date'].astype(np.datetime64)
    dataset = df.set_index('date')
    feature_names = [k for k in dataset.columns if k != task]
    
    dfp = dfg.copy()
    y_true = np.asarray(dfp.y_true_test.values[0])
    y_pred = np.asarray(dfp.y_pred_test.values[0])
    dates = np.asarray([np.datetime64(x) for x in dfp.idx_test.values[0]])
    idx = np.argsort(dates)
    
#     y_true_train = np.asarray(dfp.y_true_train.values[0])
    y_true_train = df1['value_'+task].values
    y_pred_train = np.asarray(dfp.y_pred_train.values[0])
    dates_train = np.asarray([np.datetime64(x) for x in dfp.idx_train.values[0]])
    idx_train = np.argsort(dates_train)
    
    dates_all = np.concatenate((dates_train, dates))
    idx_all = np.argsort(dates_all)
    y_true_all = np.concatenate((y_true_train, y_true))
    y_pred_all = np.concatenate((y_pred_train, y_pred))
    ########## 
    h = plt.figure(figsize=(12,8))
#     plt.suptitle(f'{horizon} day prediction of {task} (Method: {alg})')
    
#     ax = h.add_subplot(221)
#     plt.subplot(121)
    plt.subplot(223)
#     plt.subplot(325)
#     plt.plot(dates_train[idx_train],y_pred_train[idx_train],'x',color='gray',label='y\_pred\_train')
    plt.plot(dates_train[idx_train],y_true_train[idx_train],'x',color='gray',alpha=0.5,label='y\_true\_train')
#     plt.plot(dates_all[idx_all],y_true_all[idx_all],'.',color='r',alpha=0.3,label='y\_true\_all')
    
    plt.plot(dates[idx],y_true[idx],'xr',label='y\_true\_test')
    plt.plot(dates_all[idx_all],y_pred_all[idx_all],'-',color='b',label='y\_pred\_all')
#     plt.plot(dates[idx],y_pred[idx],'+b',label='y\_pred')
    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.legend()
    plt.title('Predictions on Test Set')
    
#     ax = h.add_subplot(122)
    plt.subplot(224)
#     plt.subplot(326)
#     error = y_true-y_pred
    plt.plot(y_pred,y_true,'.b',label=  f'$R^2$={r2_score(y_true, y_pred):.3f}')
    plt.ylabel('y\_true')
    plt.xlabel('y\_pred')
    plt.plot([y_true.min(),y_true.max()],[y_true.min(),y_true.max()],'--k') 
    plt.legend()
    
#     plt.subplot(311)
    plt.subplot(211)
    ax = plt.gca()
#     bbox = ax.get_tightbbox(h.canvas.get_renderer())
    mdl = dfg.simplified_model.values[0]
    mdl = mdl.replace('value_','')
    simp = dfg.simplicity.values[0]
#     simp = simplicity(str(mdl), feature_names)
    max_size=1000
    max_w=700
#     if isinstance(mdl,float):
#         mdl = dfg.symbolic_model.values[0]
        
    if len(mdl) < max_size: 
# #         print('before parsing:',mdl)
        mdl = sp.parse_expr(mdl)
#         print('before rounding:',mdl)
        tmp = round_floats(mdl,dec=5)
        tmp = round_floats(tmp,dec=5)
#         print('after rounding:',tmp)
        if not isinstance(tmp,float):
            mdl = tmp
#             print('changes accepted')
        mdl = sp.latex(mdl, 
                       fold_short_frac = True, 
    #                    mode='inline',
    #                        max=3
                      )
    #     if len(mdl) > max_size:
    #         mdl = mdl[:max_size] + ' \cdots $'
        mdl = task + ' = ' + mdl
        mdl = (
#                r'\begin{tabularx}{'+str(max_w) + r'pt}{p{'+str(max_w) + r'pt}} '
    #            r'\begin{tabular}{p{20pt}} '
               '$' + mdl + '$'
#                +r' \end{tabularx}'
              )
    
    else: 
        mdl = 'black-box'
        
    print(mdl,'len:',len(mdl))
    dfx = pd.DataFrame({
                       'Task':f'Predict {task}',
                       'Horizon':f'{horizon} days',
                       'Algorithm':alg,
                       'R2 Score':dfg.r2_test.values[0],
                       'Simplicity':simp,
                       'Model':mdl,
#                        'Model':'',
                      },
                      index=[0]
                     ).round(3) #.T #.rest_index()
    
    ax.axis('off')
    table = plt.table(cellText=dfx.values.T, 
                      rowLabels=dfx.columns, 
#                       loc='upper center', 
#                       loc='bottom', 
                      bbox=[0.0,0.0,1.0,0.7],
#                       bbox=bbox,
                      rowLoc='right',
                      cellLoc='left',
                      edges='open',
                      colWidths = [1],
                     ) 
    table.auto_set_font_size(False)
    table.set_fontsize(16)
    for key,cell in table._cells.items():
        if key == (5,0):
#             print('old model cell height:',cell._height)
            cell._height *= 2
#             print('new model cell height:',cell._height)
#             cell._size=11
            if len(mdl) < 200:
                cell.set_fontsize(14)
            elif len(mdl) < 500:
                cell.set_fontsize(10)
            elif len(mdl) < 1000:
                cell.set_fontsize(9)
            else:
                cell.set_fontsize(8)
        else:
            ahh = cell._height
#             cell._size=14
#             print('cell height:',h)
            cell._height = 4/5*ahh
    h.tight_layout()
    for ext in ['.pdf','.png']:
        h.savefig(f'figs/{task}_{horizon}_{alg}_summary.{ext}', dpi=300,
                 bbox_inches='tight')

In [ ]:
i = 0
for (task,horizon,alg),dfg in df_best.groupby(['task','horizon','algorithm']):
#     if i == 9: break
    i+=1
    if horizon == 7: continue
    comparison_plot(task,horizon,alg,dfg)
#     mdl = dfg.simplified_model.values[0]
#     mdl = sp.parse_expr(mdl)
#     mdl = sp.latex(mdl)
#     display(mdl)

In [ ]:
# from sympy import init_printing
# from sympy.printing.dot import dotprint
# import warnings
# pd.options.display.max_colwidth = 2000
# # import graphviz
# # init_printing()
# # from IPython.core.display import display, HTML
# # display(HTML("<style>.container { width:100% !important; }</style>"))
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
#     for (task,horizon,alg),dfg in df_best.groupby(['task','horizon','algorithm']):
#         if horizon == 7: continue
#     #     comparison_plot(task,horizon,alg,dfg)
#         print(40*'=')
#         print(task,horizon,alg)
#         mdl = dfg.simplified_model.values[0]
#         simplicity = dfg.simplicity.values[0]
#         max_size=2000
#         max_w=50
#         if isinstance(mdl,float):
#             mdl = dfg.symbolic_model.values[0]
#         else:
#             mdl=mdl.replace('value_','')
#             if len(mdl) < max_size: 
#         #         print('before parsing:',mdl)
#                 mdl = sp.parse_expr(mdl)
#         #         print('before rounding:',mdl)
#                 mdl = round_floats(mdl)
#                 mdl = round_floats(mdl)
#         #         print('after rounding:',mdl)
#         #         mdl = sp.latex(mdl, max=3)
#         #         mdl = f'${mdl}$'
#         #         mdl = str(mdl)
# #         if isinstance(mdl,float):
# #             mdl = dfg.symbolic_model.values[0]

#     #     w = max_w
#     #     step = max_w
#     #     while w < len(mdl):
#     #         mdl = mdl[:w] + '\n' + mdl[w:]
#     #         w += step

#     #     if len(mdl) > max_size:
#     #         mdl = 'black-box'
#         df = pd.DataFrame({'Task':f'{horizon}-day prediction of {task}',
#                            'R2 Score':dfg.r2_test.values[0],
#                            'Simplicity':simplicity,
#     #                        'Model':mdl
#                           },
#                           index=[alg]
#                          ).round(3).T
#     #     display(df)
#         display(df)
#         print('symbolic model:')
#         display(dfg.symbolic_model.values[0])
#         print('model:')
#         display(mdl)
#     #     display(mdl)
#         print(40*'=')